In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install dotenv
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
th

In [3]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

class Utils:
    def __init__(self):
        pass

    def create_dlai_index_name(self, index_name):
        """Create a unique index name based on the Gemini API key (or part of it)."""
        gemini_key = ''
        if self.is_colab():
            from google.colab import userdata
            gemini_key = userdata.get("GEMINI_API_KEY")
        else:
            gemini_key = os.getenv("GEMINI_API_KEY")

        # Use a hashed or masked portion of the key if needed
        suffix = gemini_key[-12:].lower().replace("_", "-") if gemini_key else "no-key"
        return f'{index_name}-{suffix}'

    def is_colab(self):
        """Detect if the code is running in Google Colab."""
        return 'google.colab' in sys.modules

    def get_gemini_api_key(self):
        """Load Gemini API key from environment or .env file."""
        _ = load_dotenv(find_dotenv())
        return os.getenv("GEMINI_API_KEY")

    def get_pinecone_api_key(self):
        """Load Pinecone API key from environment or .env file."""
        _ = load_dotenv(find_dotenv())
        return os.getenv("PINECONE_API_KEY")

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
pinecone_api_key = user_secrets.get_secret("PINECONE_API_KEY")

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm, trange

import pandas as pd
import time
import os

In [6]:
!wget -q --show-progress -O all-the-news-3.zip "https://www.dropbox.com/scl/fi/wruzj2bwyg743d0jzd7ku/all-the-news-3.zip?rlkey=rgwtwpeznbdadpv3f01sznwxa&dl=1"

!unzip all-the-news-3.zip

all-the-news-3.zip  100%[===================>] 166.04M  78.5MB/s    in 2.1s    
Archive:  all-the-news-3.zip
  inflating: all-the-news-3.csv      


In [8]:
with open('/kaggle/working/all-the-news-3.csv', 'r') as f:
    header = f.readline()
    print(header)

date,year,month,day,author,title,article,url,section,publication



In [9]:
df = pd.read_csv('/kaggle/working/all-the-news-3.csv', nrows=99)
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [11]:
# get api key
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [12]:
pinecone = Pinecone(api_key=pinecone_api_key)

utils = Utils()
INDEX_NAME = utils.create_dlai_index_name('dl-ai')
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=768, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("jinaai/jina-embeddings-v2-base-en", trust_remote_code=True)

def get_embeddings(articles, model=model):
   return model.encode(articles, convert_to_numpy=True).tolist()

2025-09-05 15:39:36.751572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757086776.974565      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757086777.032564      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
CHUNK_SIZE=400
TOTAL_ROWS=10000
progress_bar = tqdm(total=TOTAL_ROWS)
chunks = pd.read_csv('/kaggle/working/all-the-news-3.csv', chunksize=CHUNK_SIZE, 
                     nrows=TOTAL_ROWS)
chunk_num = 0
for chunk in chunks:
    titles = chunk['title'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{
        'id':str(chunk_num*CHUNK_SIZE+i), 
        'values':embeddings[i],
        'metadata':{'title':titles[i]},} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
      index.upsert(prepped)
      prepped = []
    progress_bar.update(len(chunk))

  0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [16]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000,
 'vector_type': 'dense'}

## Build Recommender System

In [23]:
def get_recommendations(pinecone_index, search_term, top_k=50):
  embed = get_embeddings([search_term])
  res = pinecone_index.query(vector=embed, top_k=top_k, include_metadata=True)
  return res

In [24]:
reco = get_recommendations(index, 'obama')
for r in reco.matches:
    print(f'{r.score} : {r.metadata["title"]}')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.792855263 : MUNCHIES: The Podcast: Drinking Beers with President Obama’s White House Chef
0.776218414 : Obama has a good record to brag about tonight, but people still hate the government
0.794696808 : Obama meets with national security team on Syria, Islamic State
0.802117348 : Michelle Obama Hits SoulCycle
0.799660683 : Watch President Obama dance the tango in Argentina
0.798002243 : Obama picks Merrick Garland for Supreme Court
0.782594681 : Obama to Democrats: 'I want us to run scared' in election
0.784536421 : Derek Jeter -- Real Talk from Obama ... 'For a Baseball Player, You Were Old!' (VIDEO)
0.791374207 : Obama posed with Louis Farrakhan in old photo
0.787650168 : President Obama: Michelle & I Are Gonna Be Renters
0.775066376 : Goodbye to all that: what we’ve learned from Obama’s presidency
0.762540817 : Barack Obama in talks to create shows for Netflix: New York Times
0.766191542 : The real target of Trump’s inaugural speech wasn’t Barack Obama. It was George W. Bush.
0.763

### 2.  Create Embeddings of All News Content